###Initialise the workspace

In [1]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

Azure ML SDK Version:  1.20.0
mldeeplearning	eastus2	rgdeeplearning	eastus2


###Initialize an Experiment

In [2]:
experiment_name = 'langdetection'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

###Create a Datastore/Dataset

In [4]:
import os
from azureml.core import Datastore, Dataset
script_folder = os.path.join(os.getcwd(), "AzureMLFramework")
os.makedirs(script_folder, exist_ok=True)
'''
datastore_name='workspaceblobstore'
datastore = Datastore.get(ws, datastore_name)
#Upload Data
'''
ds = ws.get_default_datastore()
#ds.upload(src_dir='./src', target_path='dataset.csv', overwrite=True, show_progress=True)
paths=[(ds, 'dataset.csv')]
path='https://raw.githubusercontent.com/sonalgan/DeepLearning/master/dataset.csv'
lang_ds = Dataset.Tabular.from_delimited_files(path=path)
'''
ds = ws.get_default_datastore()
#weather_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)
print(ds.datastore_type, ds.account_name, ds.container_name)
ds.upload(src_dir='./src', target_path='langdataset.csv', overwrite=True, show_progress=True)
'''
lang_ds.take(3).to_pandas_dataframe()

,words,lang
0,तुम,hi
1,हम,hi
2,रूप,hi


In [5]:
df = lang_ds.to_pandas_dataframe()
df.tail()

,words,lang
99995,ਰੋਗਾਣੂ,pa
99996,ਸਰਜਨ,pa
99997,ਟਾਇਰਾਂ,pa
99998,ਪੀਐੱਮਜੀਕੇਪੀ,pa
99999,ਰੁੱਝੀ,pa


In [28]:
df=pd.read_csv('./dataset.csv',encoding='utf-8')
df.head()

,words,lang
0,तुम,hi
1,हम,hi
2,रूप,hi
3,लोगों,hi
4,मैं,hi


### Register a dataset

In [9]:
lang_ds = lang_ds.register(workspace=ws,
                               name='lang_ds',
                               description='language training data')

###Create a compute cluster

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "AzureMLFramework"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D16s_v3',
                                                           max_nodes=4,
                                                          vm_priority='dedicated')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


###Environment Set Up

In [24]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

fwrk = Environment("dlenv")

fwrk.docker.enabled = True
fwrk.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn',
                                                                          'pandas',
                                                                          'numpy',
                                                                          'tensorflow==1.15',
                                                                          'keras',
                                                                          'matplotlib'
                            
                                                                         ])

fwrk.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
fwrk.python.conda_dependencies.add_pip_package('joblib')
fwrk.python.conda_dependencies.add_pip_package('wget')
fwrk.python.conda_dependencies.save_to_file(".", "dlenv.yml")

'dlenv.yml'

##Train the Model

In [ ]:
###Create training script

In [ ]:
###Submit the Training Job to the Compute Cluster

In [10]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
lang_ds=Dataset.get_by_name(workspace=ws, name='lang_ds')
args=['--input-data', lang_ds.as_named_input('lang'),
'--batch-size',1024,
'--test-size',0.3,
'--first-layer-neurons',600,
'--second-layer-neurons',400,
'--third-layer-neurons',200,
'--fourth-layer-neurons',0,
'--learning-rate',0.001,
'--dropout1',0.5,
'--dropout2',0,
'--dropout3',0,
'--weight-constraint',4]

src = ScriptRunConfig(source_directory=script_folder, script='./train.py',
                      arguments=args
                     )

# Set compute target to the one created in previous step
src.run_config.target = cpu_cluster.name

# Set environment
src.run_config.environment = fwrk
 
#run = exp.submit(config=src)
#run

In [20]:
%%time
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True) 

RunId: langdetection_1613505605_6fe92740
Web View: https://ml.azure.com/experiments/langdetection/runs/langdetection_1613505605_6fe92740?wsid=/subscriptions/d7b4badf-6888-4e39-b572-64a569d1930a/resourcegroups/rghpe/workspaces/mlhpe2

Streaming azureml-logs/55_azureml-execution-tvmps_86cb858287b5197ebfa49706895cede08ca6e472e8a1e8d9c457150adaad11ae_d.txt

2021-02-16T20:03:31Z Starting output-watcher...
2021-02-16T20:03:31Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-02-16T20:03:32Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-02-16T20:03:32Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_2c82a7ea0909b66d46f5eb0620b4f0f0
be8ec4e48d7f: Pulling fs layer
33b8b485aff0: Pulling fs layer
d887158cc58c: Pulling fs layer
05895bb28c18: Pulling fs layer
baf7ab26f516: Pulling fs layer
181182e3c9cf: Pulling fs layer
d584ef274e55: Pulling fs layer
baf7ab26f516: Waiting
05895b

{'runId': 'langdetection_1613505605_6fe92740',
 'target': 'AzureMLFramework',
 'status': 'Completed',
 'startTimeUtc': '2021-02-16T20:03:26.45396Z',
 'endTimeUtc': '2021-02-16T20:19:53.79707Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'dea61172-052b-4fab-be68-ffe1faece4ec',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '49ff27a4-f9ee-4189-bb01-61ec40da3d9d'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'lang', 'mechanism': 'Direct'}}, {'dataset': {'id': '49ff27a4-f9ee-4189-bb01-61ec40da3d9d'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data',
   'DatasetConsumptionConfig:lang',
   '--batch-size',
   '1024',
   '--first-layer-neurons',
   '600',
   '--second-layer-neurons',
   '400',
   '--thi

### Register the Model

In [21]:
model = run.register_model(model_name='model', model_path='./outputs/model.pkl')
print(model.name, model.id, model.version, sep='\t')

from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config()
model=Model(ws, 'model')

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "./model.pkl")

os.stat(file_path)

model	model:1	1


os.stat_result(st_mode=33279, st_ino=18014505162109878272, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=9189325, st_atime=1613512727, st_mtime=1613512727, st_ctime=1613512727)

## Hyperparameter Tuning

###Sampling the hyperparameter space

In [37]:
from azureml.train.hyperdrive import BayesianParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform, quniform,uniform

ps =BayesianParameterSampling(
    {
        '--batch-size': choice(32, 64, 128,256,512,1024),
        '--test-size':choice(0.10,0.15,0.20),
        '--first-layer-neurons': choice( 600, 650, 700),
        '--second-layer-neurons': choice( 400, 450, 500),
        '--third-layer-neurons': choice(0,200,250,300),
        '--fourth-layer-neurons': choice(0,50,100),
        '--learning-rate': choice(0.001,0.0001,0.00001,0.000001),
        '--dropout1': choice(0,0.1,0.2,0.3,0.4,0.5),
        '--dropout2': choice(0,0.1,0.2),
        '--dropout3':choice(0,0.1,0.2),
        '--weight-constraint':choice(0,1,2,3,4)
    }
)




###Create Config scipt

In [38]:
args = ['--input-data', lang_ds.as_named_input('lang')]

src = ScriptRunConfig(source_directory=script_folder, script='./train.py',
                      arguments=args
                     )
src.run_config.target = cpu_cluster.name
src.run_config.environment = fwrk

###Define Termination policy

In [55]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [37]:
###Specify primary metric

In [39]:
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Loss',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     max_total_runs=220,
                                     max_concurrent_runs=4)

###Submit the Hypertuning job to the compute cluster

In [40]:
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [41]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_f2d3ca9c-bcef-4cbd-878c-10b6d18046bb
Web View: https://ml.azure.com/experiments/langdetection/runs/HD_f2d3ca9c-bcef-4cbd-878c-10b6d18046bb?wsid=/subscriptions/2178c198-cafd-42e8-9039-65851f5de86f/resourcegroups/rgdeeplearning/workspaces/mldeeplearning

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-17T05:48:13.199073][API][INFO]Experiment created<END>\n""<START>[2021-02-17T05:48:13.806051][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-17T05:48:14.086398][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-17T05:48:14.9837540Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [43]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [42]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--input-data', 'DatasetConsumptionConfig:lang', '--batch-size', '512', '--dropout1', '0.7', '--dropout2', '0.7', '--dropout3', '0.5', '--first-layer-neurons', '350', '--fourth-layer-neurons', '400', '--learning-rate', '0.002537527700592502', '--second-layer-neurons', '600', '--test-size', '0.1', '--third-layer-neurons', '500', '--weight-constraint', '4']


In [46]:


print(best_run.get_file_names())



['azureml-logs/55_azureml-execution-tvmps_36cd85af4a51758a4dc2c981f4ecdb1aa5b31c1e9d1307483a4dd9402216a9b6_d.txt', 'azureml-logs/65_job_prep-tvmps_36cd85af4a51758a4dc2c981f4ecdb1aa5b31c1e9d1307483a4dd9402216a9b6_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_36cd85af4a51758a4dc2c981f4ecdb1aa5b31c1e9d1307483a4dd9402216a9b6_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_56145ac0-043b-465f-a017-798a7d3737d3.jsonl', 'logs/azureml/dataprep/python_span_l_56145ac0-043b-465f-a017-798a7d3737d3.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.pkl']


In [45]:
model = best_run.register_model(model_name='langmodel', model_path='./outputs/model')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_36cd85af4a51758a4dc2c981f4ecdb1aa5b31c1e9d1307483a4dd9402216a9b6_d.txt', 'azureml-logs/65_job_prep-tvmps_36cd85af4a51758a4dc2c981f4ecdb1aa5b31c1e9d1307483a4dd9402216a9b6_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_36cd85af4a51758a4dc2c981f4ecdb1aa5b31c1e9d1307483a4dd9402216a9b6_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_56145ac0-043b-465f-a017-798a7d3737d3.jsonl', 'logs/azureml/dataprep/python_span_l_56145ac0-043b-465f-a017-798a7d3737d3.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.pkl']
          